# Aplicaciones con Pubsub

## Requisitos previos a la clase

### Instalar Docker Desktop y Configurar una Cuenta en Docker Hub


*   Descarga e instala Docker Desktop desde el [sitio web oficial de Docker](https://docs.docker.com/engine/install/).

*   Una vez instalado, crea o inicia sesión en tu cuenta de Docker Hub.

*   Abre Docker Desktop e inicia sesión con tus credenciales de Docker Hub.

### Instalar Cloud SDK Emulators y Configurar un emulador de PubSub

*   Ejecuta el siguiente comando en tu terminal para descargar la imagen Docker:
    ```
    docker pull google/cloud-sdk:emulators
    ```

*   Ejecuta el siguiente comando en tu terminal paraejecutar el contenedor Docker:
    ```
    docker run --rm -p 8085:8085 google/cloud-sdk:emulators /bin/bash -c "gcloud beta emulators pubsub start --project=testing-pubsub --host-port='0.0.0.0:8085'"
    ```
Esto iniciará un contenedor con:
  *   El puerto expuesto en 8085 (localhost:8085) — el puerto predeterminado es el puerto 8085 para IPv6.
  *   El contenedor se basará en la imagen google/cloud-sdk:emulators.
  *   Ejecutará un comando de inicio para el emulador de Pub/Sub.
  *   El ID del proyecto es el que utilizarás más adelante, localmente, no es tu ID de proyecto real.

*   Clona el repositorio de Google PubSub
    ```
    git clone https://github.com/googleapis/python-pubsub.git
    ```

*   Navega al directorio de los snippets:
    ```
    cd python-pubsub/sample/snippets
    ```

*   Crea un ambiente virtual
    ```
    python3 -m venv env
    ```

*   Activa el ambiente virtual
  *   Activación en Unix
    ```
    source env/bin/activate
    ```
  *   Activación en Windows
    ```
    env\Scripts\activate
    ```

*   Instala los requisitos
    ```
    pip3 install -r requirements.txt
    ```

*   Exporta la variable PUBSUB_EMULATOR_HOST y PUBSUB_PROJECT_ID
    ```
    export PUBSUB_EMULATOR_HOST=localhost:8085
    export PUBSUB_PROJECT_ID=testing-pubsub
    ```

### Prueba que todo este correcto

*   Crea un tópico
    ```
    python publisher.py testing-pubsub create testing-topic
    ```

*   Crea una suscripción a ese topico
    ```
    python subscriber.py testing-pubsub create testing-topic testing-subscription
    ```

*   Empieza a escuchar a ese tópico
    ```
    python subscriber.py testing-pubsub receive testing-subscription
    ```
  
*   Abre otra terminal, exporta la variable PUBSUB_EMULATOR_HOST y PUBSUB_PROJECT_ID y publica al tópico
    ```
    python publisher.py testing-pubsub publish testing-topic
    ```

Si todo esta correcto, tu terminal que escucha al tópico deberia verse algo asi:
```
Listening for messages on projects/testing-pubsub/subscriptions/testing-subscription..

Received Message {
  data: b'Message number 1'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'Message number 2'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'Message number 3'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'Message number 4'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'Message number 5'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'Message number 6'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'Message number 7'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'Message number 8'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'Message number 9'
  ordering_key: ''
  attributes: {}
}.
```

## Construir un API REST con Django REST Framework

Crea un ambiente virtual:
```
python3 -m venv env
```

Activa el ambiente virtual:
```
# Activación en Unix
source env/bin/activate

# Activación en Windows
env\Scripts\activate
```

Instala Django, DRF y PyJWT:
```
pip install django
pip install djangorestframework
pip install google-cloud-pubsub
```

Crea un nuevo proyecto en Django:
```
django-admin startproject stock_alert_system
```

Crea una nueva aplicación en Django:
```
python manage.py startapp api
```

Agrega la aplicación de `rest_framework` y la que acabamos de crear en el archivo de `settings.py`:
```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'api'
]
```

Genera las migraciones y ejeculatas
```
python manage.py makemigrations
python manage.py migrate
```

Crea un super usuario
```
python manage.py createsuperuser
```

Corre la aplicación para corroborar que todo esta correcto
```
python manage.py runserver
```

## Construir un CRUD de Items

### Define la estructura de datos para un modelo Item

In [ ]:
# api/models.py
# Importa el módulo models de Django, contiene funcionalidades esenciales para definir modelos en Django.
from django.db import models


# Define una nueva clase Item, esto indica que es una tabla de base de datos.
class Item(models.Model):
    # Define un campo de texto name para el modelo Item.
    name = models.CharField(max_length=100)
    # Define un campo de texto description para el modelo Item.
    description = models.TextField()

    # Define un administrador de modelo, es la interfaz predeterminada de Django para consultas de base de datos.
    objects = models.Manager()

    # Este método determina cómo se representa una instancia del modelo Item como cadena de texto.
    def __str__(self):
        # Devuelve el valor del campo name.
        return self.name

### Crea el serializer para un modelo Item

In [ ]:
# api/serializers.py
# Importa ModelSerializer desde rest_framework.serializers, es utilizado para construir el serializador.
from rest_framework.serializers import ModelSerializer

# Importa la clase Item desde el módulo de modelos.
from .models import Item


# Define una clase ItemSerializer, esto genera automáticamente campos y métodos de validación basados en el modelo.
class ItemSerializer(serializers.ModelSerializer):
    # Se utiliza para configurar el serializer.
    class Meta:
        # Especifica el modelo que el serializer manejará.
        model = Item
        # Define los campos que deben incluirse en la serialización y deserialización.
        fields = ["id", "name", "description"]

### Define las vistas CRUD para Items

In [ ]:
# api/views.py
# Importa ModelViewSet desde rest_framework.viewsets, es una clase que proporciona acciones completas CRUD para un modelo Django en particular.
from rest_framework.viewsets import ModelViewSet

# Importa la clase Item desde el módulo de modelos.
from .models import Item
# Importa la clase ItemSerializer desde el módulo de serializers.
from .serializers import ItemSerializer


# Define una clase ItemViewSet proporciona la implementación de los métodos necesarios para las operaciones CRUD.
class ItemViewSet(ModelViewSet):
    # Establece el queryset que el viewset usará para operaciones de base de datos.
    queryset = Item.objects.all()
    # Asigna la clase ItemSerializer como el serializer_class del viewset.
    serializer_class = ItemSerializer

### Configura las URLs de la aplicación

In [ ]:
# api/urls.py
# Importa la función path de Django, esta función es utilizada para definir patrones de URL individuales.
from django.urls import path

# Importa ItemViewSet desde el módulo de vistas.
from .views import ItemViewSet

# Define una lista de rutas URL llamada urlpatterns.
urlpatterns = [
    # Define la ruta para las operaciones de listado y creación.
    path("items/", ItemViewSet.as_view({"get": "list", "post": "create"})),
    # Define la ruta para operaciones individuales en un Item específico.
    path(
        "items/<int:pk>/",
        ItemViewSet.as_view(
            {"get": "retrieve", "patch": "partial_update", "delete": "destroy"}
        ),
    ),
]

### Configura las URLs del proyecto

In [ ]:
# stock_alert_system/urls.py
# Importa el módulo admin de Django, proporciona el sitio de administración automático de Django.
from django.contrib import admin
# Importa include y path desde django.urls.
# path se utiliza para definir patrones de URL en la aplicación.
# include se usa para hacer referencia a otras configuraciones de URL en el proyecto.
from django.urls import path, include

# Define una lista llamada urlpatterns
urlpatterns = [
    # Define una ruta para el sitio de administración.
    path("admin/", admin.site.urls),
    # Define una ruta para incluir otras configuraciones de URL.
    path("api/", include("api.urls")),
]

## Construir sistema de Stock

### Define la estructura de datos para un modelo Stock

In [ ]:
# api/models.py
# Importa MinValueValidator desde django.core.validators, esto permite asegurar que un campo numérico no caiga por debajo de un valor mínimo especificado.
from django.core.validators import MinValueValidator
# Importa el módulo models de Django, contiene funcionalidades esenciales para definir modelos en Django.
from django.db import models

# Define una nueva clase Stock, esto indica que es una tabla de base de datos.
class Stock(models.Model):
    # Define un campo item como un OneToOneField, esto establece una relación uno a uno entre Stock e Item.
    # on_delete=models.CASCADE especifica que si el Item relacionado se elimina, la instancia de Stock correspondiente también se eliminará.
    # related_name="stock" permite acceder al objeto Stock relacionado desde una instancia de Item usando item.stock.
    # primary_key=True indica que este campo también servirá como clave primaria para la tabla Stock, lo que significa que cada Item solo puede tener una instancia de Stock asociada.
    item = models.OneToOneField(
        Item, on_delete=models.CASCADE, related_name="stock", primary_key=True
    )
    # Define un campo quantity, este campo almacena un entero que representa la cantidad de stock disponible para el Item relacionado.
    # validators=[MinValueValidator(1)] asegura que el valor mínimo que puede tener quantity es 1, evitando valores negativos o cero.
    quantity = models.IntegerField(validators=[MinValueValidator(1)])

    # Define un administrador de modelo, es la interfaz predeterminada de Django para consultas de base de datos.
    objects = models.Manager()

    # Este método determina cómo se representa una instancia del modelo Item como cadena de texto.
    def __str__(self):
        # Devuelve el valor del campo name y el campo stock.
        return f"{self.item.name} - Stock: {self.quantity}"

### Crea el serializer para un modelo Stock

In [ ]:
# api/serializers.py
# Importa ChoiceField, ModelSerializer y ValidationError desde rest_framework.serializers, estas clases y excepciones son utilizadas para construir el serializador y gestionar la validación.
from rest_framework.serializers import (ChoiceField, ModelSerializer,
                                        ValidationError)

 # Importa la clase Stock desde el módulo de modelos.
from .models import Stock


# Define una clase StockSerializer, esto genera automáticamente campos y métodos de validación basados en el modelo.
class StockSerializer(ModelSerializer):
    # Agrega un campo adicional action que solo acepta valores predefinidos ("add" y "subtract").
    # Es write_only=True, lo que indica que este campo se utiliza solo para recibir datos en operaciones de escritura.
    action = ChoiceField(choices=["add", "subtract"], write_only=True)

    # Se utiliza para configurar el serializer.
    class Meta:
        # Especifica el modelo que el serializer manejará.
        model = Stock
        # Define los campos que deben incluirse en la serialización y deserialización.
        fields = ["item", "quantity", "action"]

    # Este método se llama automáticamente cuando se valida el serializador.
    def validate(self, data):
        # Extrae action y quantity de data, data es el diccionario con los datos enviados para la serialización.
        action = data.get("action")
        quantity = data.get("quantity", 0)

        # Si la acción es "subtract" y ya existe una instancia del modelo, se verifica si la cantidad en stock es suficiente para ser reducida.
        # Si no es suficiente, se lanza una excepción ValidationError con un mensaje apropiado.
        if action == "subtract" and self.instance:
            if self.instance.quantity < quantity:
                raise ValidationError(
                    "You cannot remove more stock than what is available."
                )
        # Si los datos pasan todas las validaciones, se devuelven.
        return data

### Crea el Signal que genere el stock vacio de un Item

In [ ]:
# api/signals.py
# Se importa la señal post_save, que es emitida por Django después de guardar un objeto en la base de datos.
from django.db.models.signals import post_save
# Es un decorador que se usa para registrar una función como receptor de una señal.
from django.dispatch import receiver

# Importa los modelos Item y Stock del mismo paquete.
from .models import Item, Stock


# Este decorador registra la función create_stock como un receptor de la señal post_save
@receiver(post_save, sender=Item)
# sender: La clase del modelo que envió la señal.
# instance: La instancia del modelo que fue guardada.
# created: Un booleano que es True si la instancia fue creada (en lugar de actualizada).
def create_stock(sender, instance, created, **kwargs):
    # Verifica si la instancia de Item fue creada durante esta operación de guardado.
    if created:
        # Si el Item es nuevo, se crea un nuevo objeto Stock asociado con este Item.
        Stock.objects.create(item=instance, quantity=0)

### Configura los Signals

In [ ]:
# api/apps.py
# Importa la clase base AppConfig del módulo django.apps. Esta clase se usa para configurar ciertas características de las aplicaciones Django.
from django.apps import AppConfig


# Define una nueva clase de configuración llamada StockManagementConfig.
class StockManagementConfig(AppConfig):
    # Esta línea configura el tipo de campo automático predeterminado que se utilizará para los campos de clave primaria en los modelos de esta aplicación.
    default_auto_field = "django.db.models.BigAutoField"
    # Define el nombre de la aplicación dentro de Django.
    name = "api"

    # Define un método que se llama cuando Django ha cargado completamente la aplicación, incluyendo todos los modelos.
    def ready(self):
        # Dentro del método ready, se importan las señales desde el módulo local (signals.py).
        # Esta importación asegura que cualquier señal definida en el módulo signals se conecte adecuadamente cuando la aplicación esté lista.
        from . import signals

### Define las vista de actualización para Stock

In [ ]:
# api/views.py
# Este módulo proporciona un conjunto de códigos de estado HTTP codificados simbólicamente.
from rest_framework import status
# Esta clase extiende la clase de solicitud HTTP estándar de Django para proporcionar más funcionalidades que son útiles en el contexto de APIs.
from rest_framework.request import Request
# La clase Response es una subclase de HttpResponse de Django que se utiliza para devolver respuestas en APIs construidas con DRF.
from rest_framework.response import Response
# Importa ModelViewSet desde rest_framework.viewsets, es una clase que proporciona acciones completas CRUD para un modelo Django en particular.
from rest_framework.viewsets import ModelViewSet

# Importa la clase Stock desde el módulo de modelos.
from .models import Stock
# Importa la clase StockSerializer desde el módulo de serializers.
from .serializers import  StockSerializer


# Define una clase StockViewSet proporciona la implementación de los métodos necesarios para las operaciones CRUD.
class StockViewSet(ModelViewSet):
    # Establece el queryset que el viewset usará para operaciones de base de datos.
    queryset = Stock.objects.all()
    # Asigna la clase StockSerializer como el serializer_class del viewset.
    serializer_class = StockSerializer

    # Este método se llama cuando se realiza una solicitud PATCH a la API.
    def partial_update(self, request: Request, *args, **kwargs):
        # Obtiene la instancia del modelo Stock que se está actualizando.
        instance = self.get_object()
        # Crea un serializer para la instancia con los datos proporcionados en la solicitud.
        serializer = self.get_serializer(instance, data=request.data, partial=True)
        # Verifica si los datos son válidos. Si hay errores, lanza una excepción.
        if not serializer.is_valid(raise_exception=True):
            return Response(
                {"status": "BAD_REQUEST", "data": serializer.errors},
                status=status.HTTP_400_BAD_REQUEST,
            )

        # Obtiene la acción y la cantidad de los datos validados por el serializer.
        action = serializer.validated_data.get("action")
        quantity = serializer.validated_data.get("quantity")

        # Dependiendo de la acción especificada, actualiza el atributo quantity de la instancia. Incrementa para "add" y decrementa para "subtract".
        if action == "add":
            instance.quantity += quantity
        elif action == "subtract":
            instance.quantity -= quantity

        # Guarda los cambios en la base de datos.
        instance.save()

        # Obtiene un nuevo serializer para la instancia actualizada para preparar los datos para la respuesta.
        updated_serializer = self.get_serializer(instance)
        # Construye y devuelve una respuesta con el estado HTTP 200 OK y los datos serializados de la instancia actualizada.
        return Response(
            {"status": "OK", "data": updated_serializer.data},
            status=status.HTTP_200_OK,
        )

### Configura las URLs de la aplicación

In [ ]:
# api/urls.py
# Importa la función path de Django, esta función es utilizada para definir patrones de URL individuales.
from django.urls import path

# Importa ItemViewSet desde el módulo de vistas.
from .views import ItemViewSet

# Define una lista de rutas URL llamada urlpatterns.
urlpatterns = [
    # Define la ruta para las operaciones de listado y creación.
    path("items/", ItemViewSet.as_view({"get": "list", "post": "create"})),
    # Define la ruta para operaciones individuales en un Item específico.
    path(
        "items/<int:pk>/",
        ItemViewSet.as_view(
            {"get": "retrieve", "patch": "partial_update", "delete": "destroy"}
        ),
    ),
    # Define la ruta para las operaciones de listado.
    path("stock/", StockViewSet.as_view({"get": "list"})),
    # Define la ruta para operaciones individuales en un Stock específico.
    path("stock/<int:pk>/", StockViewSet.as_view({"patch": "partial_update"})),
]

### Actualiza la vista del Stock para publicar cuando un stock este bajo

In [ ]:
# api/views.py
# Clase de excepción que se utiliza para encapsular errores generados por las APIs de Google Cloud.
from google.api_core.exceptions import GoogleAPIError
# Importa el cliente de Google Cloud Pub/Sub para interactuar con el servicio de mensajería.
from google.cloud import pubsub_v1
# Este módulo proporciona un conjunto de códigos de estado HTTP codificados simbólicamente.
from rest_framework import status
# Esta clase extiende la clase de solicitud HTTP estándar de Django para proporcionar más funcionalidades que son útiles en el contexto de APIs.
from rest_framework.request import Request
# La clase Response es una subclase de HttpResponse de Django que se utiliza para devolver respuestas en APIs construidas con DRF.
from rest_framework.response import Response
# Importa ModelViewSet desde rest_framework.viewsets, es una clase que proporciona acciones completas CRUD para un modelo Django en particular.
from rest_framework.viewsets import ModelViewSet

# Importa la clase Stock desde el módulo de modelos.
from .models import Stock
# Importa la clase StockSerializer desde el módulo de serializers.
from .serializers import  StockSerializer


# Define una clase StockViewSet proporciona la implementación de los métodos necesarios para las operaciones CRUD.
class StockViewSet(ModelViewSet):
    # Establece el queryset que el viewset usará para operaciones de base de datos.
    queryset = Stock.objects.all()
    # Asigna la clase StockSerializer como el serializer_class del viewset.
    serializer_class = StockSerializer
    # Crea una instancia del cliente de Pub/Sub para ser utilizada en la publicación de mensajes.
    publisher = pubsub_v1.PublisherClient()

    # Este método se llama cuando se realiza una solicitud PATCH a la API.
    def partial_update(self, request: Request, *args, **kwargs):
        # Obtiene la instancia del modelo Stock que se está actualizando.
        instance = self.get_object()
        # Crea un serializer para la instancia con los datos proporcionados en la solicitud.
        serializer = self.get_serializer(instance, data=request.data, partial=True)
        # Verifica si los datos son válidos. Si hay errores, lanza una excepción.
        if not serializer.is_valid(raise_exception=True):
            return Response(
                {"status": "BAD_REQUEST", "data": serializer.errors},
                status=status.HTTP_400_BAD_REQUEST,
            )

        # Obtiene la acción y la cantidad de los datos validados por el serializer.
        action = serializer.validated_data.get("action")
        quantity = serializer.validated_data.get("quantity")

        # Dependiendo de la acción especificada, actualiza el atributo quantity de la instancia. Incrementa para "add" y decrementa para "subtract".
        if action == "add":
            instance.quantity += quantity
        elif action == "subtract":
            instance.quantity -= quantity

        # Guarda los cambios en la base de datos.
        instance.save()

        # Obtiene un nuevo serializer para la instancia actualizada para preparar los datos para la respuesta.
        updated_serializer = self.get_serializer(instance)

        # Asegura que el código dentro del bloque se ejecute, y si ocurre algún error (específicamente GoogleAPIError), este se capture y maneje.
        try:
            # Verifica si la cantidad del stock, después de ser actualizada, es menor que 5.
            if updated_serializer.data["quantity"] < 5:
                # Obtiene la ruta del tema específico donde se publicarán los mensajes.
                topic_path = self.publisher.topic_path("testing-pubsub", "stock")
                # Crea un diccionario con los datos que se enviarán, que incluyen el ID del artículo, el nombre y la cantidad de stock actual.
                message_json = {
                    "id": instance.item.id,
                    "name": instance.item.name,
                    "stock": instance.quantity,
                }
                # Convierte el diccionario a una cadena de texto y luego la codifica a bytes en formato UTF-8, lo cual es necesario para la publicación en Pub/Sub.
                message_data = str(message_json).encode("utf-8")
                # Publica el mensaje al tema especificado y guarda el objeto future que puede ser usado para verificar si la publicación fue exitosa.
                future = self.publisher.publish(topic_path, message_data)
                # Imprime el resultado del future, lo cual bloquea hasta que la publicación se complete y retorna el ID del mensaje publicado o lanza un error si algo sale mal.
                print(f"Future: {future.result()}")

        # Captura cualquier error que se lance durante la publicación del mensaje y lo imprime.
        except GoogleAPIError as e:
            print(f"Future error: {e}")

        # Construye y devuelve una respuesta con el estado HTTP 200 OK y los datos serializados de la instancia actualizada.
        return Response(
            {"status": "OK", "data": updated_serializer.data},
            status=status.HTTP_200_OK,
        )

### Ejecutamos PubSub Emulator

*   Ejecuta el siguiente comando en tu terminal paraejecutar el contenedor Docker:
    ```
    docker run --rm -p 8085:8085 google/cloud-sdk:emulators /bin/bash -c "gcloud beta emulators pubsub start --project=testing-pubsub --host-port='0.0.0.0:8085'"
    ```

*   Abre el repositorio de Google PubSub

*   Navega al directorio de los snippets:
    ```
    cd python-pubsub/sample/snippets
    ```

*   Activa el ambiente virtual
  *   Activación en Unix
    ```
    source env/bin/activate
    ```
  *   Activación en Windows
    ```
    env\Scripts\activate
    ```

*   Exporta la variable PUBSUB_EMULATOR_HOST y PUBSUB_PROJECT_ID
    ```
    export PUBSUB_EMULATOR_HOST=localhost:8085
    export PUBSUB_PROJECT_ID=testing-pubsub
    ```

*   Crea el tópico
    ```
    python publisher.py testing-pubsub create stock
    ```

*   Crea una suscripción a ese topico
    ```
    python subscriber.py testing-pubsub create stock stock-subscription
    ```

*   Empieza a escuchar a ese tópico
    ```
    python subscriber.py testing-pubsub receive stock-subscription
    ```

Si todo esta correcto, cada que actualices el stock tu terminal que escucha al tópico deberia verse algo asi:
```
Listening for messages on projects/testing-pubsub/subscriptions/testing-topic..

Received Message {
  data: b"{'id': 1, 'name': 'Macbook', 'stock': 2}"
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b"{'id': 1, 'name': 'Macbook', 'stock': 1}"
  ordering_key: ''
  attributes: {}
}.

```

## Construir Sistema de Stock Bajo

### Define las vista que enlista los Stock bajos

In [ ]:
# api/views.py
# Importa el cliente de Google Cloud Pub/Sub para interactuar con el servicio de mensajería.
from google.cloud import pubsub_v1
# Este módulo proporciona un conjunto de códigos de estado HTTP codificados simbólicamente.
from rest_framework import status
# Esta clase extiende la clase de solicitud HTTP estándar de Django para proporcionar más funcionalidades que son útiles en el contexto de APIs.
from rest_framework.request import Request
# La clase Response es una subclase de HttpResponse de Django que se utiliza para devolver respuestas en APIs construidas con DRF.
from rest_framework.response import Response
# Esta clase es la base para todas las vistas en DRF, proporciona la funcionalidad necesaria para crear respuestas HTTP a las solicitudes entrantes.
from rest_framework.views import APIView
# Esta función del módulo ast (Abstract Syntax Trees) de Python es utilizada para evaluar de manera segura y literal cadenas de caracteres que contienen expresiones Python.
from ast import literal_eval


# Define una clase ReceiveLowItemStockView proporciona métodos para manejar solicitudes HTTP.
class ReceiveLowItemStockView(APIView):
    # Crea una instancia del cliente de subscripción de Pub/Sub para ser utilizada en la recepción de mensajes.
    subscriber = pubsub_v1.SubscriberClient()

    # Este método se llama cuando se realiza una solicitud GET a la API.
    def get(self, request: Request, *args, **kwargs):
        # La ruta de la subscripción se construye con el nombre del proyecto y el identificador de la subscripción.
        subscription_path = self.subscriber.subscription_path(
            "testing-pubsub", "stock-subscription"
        )
        # Extrae hasta 10 mensajes desde Pub/Sub.
        response = self.subscriber.pull(
            request={"subscription": subscription_path, "max_messages": 10}
        )
        ack_ids = []
        items = []
        for received_message in response.received_messages:
            # Decodifica cada mensaje de datos desde UTF-8 a texto.
            item_data = received_message.message.data.decode("utf-8")
            # Evalúa literalmente el texto para convertirlo en objetos de Python.
            items.append(literal_eval(item_data))
            # Almacena los identificadores de acuse de recibo.
            ack_ids.append(received_message.ack_id)

        # Envía un acuse de recibo a Pub/Sub para los mensajes procesados,
        # indicando que los mensajes han sido recibidos y manejados correctamente,
        # lo cual es esencial para evitar la reentrega de los mismos mensajes.
        if ack_ids:
            self.subscriber.acknowledge(
                request={"subscription": subscription_path, "ack_ids": ack_ids}
            )

        #  Envia una respuesta HTTP 200 OK con los datos de los artículos procesados.
        return Response(
            {"status": "OK", "data": items},
            status=status.HTTP_200_OK,
        )

### Configura las URLs de la aplicación

In [ ]:
# api/urls.py
# Importa la función path de Django, esta función es utilizada para definir patrones de URL individuales.
from django.urls import path

# Importa ItemViewSet desde el módulo de vistas.
from .views import ItemViewSet

# Define una lista de rutas URL llamada urlpatterns.
urlpatterns = [
    # Define la ruta para las operaciones de listado y creación.
    path("items/", ItemViewSet.as_view({"get": "list", "post": "create"})),
    # Define la ruta para operaciones individuales en un Item específico.
    path(
        "items/<int:pk>/",
        ItemViewSet.as_view(
            {"get": "retrieve", "patch": "partial_update", "delete": "destroy"}
        ),
    ),
    # Define la ruta para las operaciones de listado.
    path("stock/", StockViewSet.as_view({"get": "list"})),
    # Define la ruta para operaciones individuales en un Stock específico.
    path("stock/<int:pk>/", StockViewSet.as_view({"patch": "partial_update"})),
    # Define la ruta para las operaciones de listado de stock bajo.
    path("low-stock/", ReceiveLowItemStock.as_view()),
]